In [1]:
%load_ext autoreload
%autoreload

In [2]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from pathlib import Path
from project_dataset import load_dataset
from code2nl.model import Seq2Seq

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from dataclasses import dataclass

@dataclass
class Args:
    model_name = "neulab/codebert-cpp"
    num_proc = 4
    batch_size = 50
    max_source_length = 512  
    max_target_length = 53 
    data_cols = ["CVE ID", "explain", "func_before"]
    save_dir = 'tf_board'
    epochs = 100
    grad_acc_steps = 4
    lr = 5e-5
    log_freq = 10
    local_rank = -1
    deepspeed = None
    fp16 = False
    lr_warmup_steps = 200
    weight_decay = 0.05
    task = "vulnerability_type"
    prefix = 'neulab'
    do_lower_case = False
    beam_size = 10
    
args = Args()

In [5]:
ds = load_dataset(args.task)

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['CVE ID', 'explain', 'func_before', 'processed_func'],
        num_rows: 3870
    })
    validation: Dataset({
        features: ['CVE ID', 'explain', 'func_before', 'processed_func'],
        num_rows: 431
    })
    test: Dataset({
        features: ['CVE ID', 'explain', 'func_before', 'processed_func'],
        num_rows: 1076
    })
})

In [7]:
df_train = ds['train']
df_train = df_train.to_pandas()

df_val = ds['validation']
df_val = df_val.to_pandas()

df_test = ds['test']
df_test = df_test.to_pandas()

In [8]:
df_train.head(3)

,CVE ID,explain,func_before,processed_func
0,CVE-2018-16066,out-of-bounds read,Node::InsertionNotificationRequest SVGStyleEl...,Node::InsertionNotificationRequest SVGStyleEle...
1,CVE-2015-2695,pointer type,spnego_gss_init_sec_context( OM_uint32 *min...,spnego_gss_init_sec_context(OM_uint32 *minor_s...
2,CVE-2011-2861,string read,void RenderThread::Init() { TRACE_EVENT_BEGIN...,void RenderThread::Init() {\n TRACE_EVENT_BEG...


In [9]:
import os

os.makedirs(args.save_dir, exist_ok=True)
os.makedirs(f'tmp_data/{args.task}', exist_ok=True)
os.makedirs(f'{args.save_dir}/{args.prefix}_{args.task}', exist_ok=True)

In [10]:
import json

df_train['code_tokens'] = df_train.func_before.apply(lambda x: x.split())
df_train['docstring_tokens'] = df_train.explain.apply(lambda x: x.split())
with open(f'tmp_data/{args.task}/train.jsonl','w') as f:
    for _, row in df_train.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

df_val['code_tokens'] = df_val.func_before.apply(lambda x: x.split())
df_val['docstring_tokens'] = df_val.explain.apply(lambda x: x.split())
with open(f'tmp_data/{args.task}/valid.jsonl','w') as f:
    for _, row in df_val.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

df_test['code_tokens'] = df_test.func_before.apply(lambda x: x.split())
df_test['docstring_tokens'] = df_test.explain.apply(lambda x: x.split())
with open(f'tmp_data/{args.task}/test.jsonl','w') as f:
    for _, row in df_test.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

In [11]:
def convert_examples_to_features(examples, tokenizer, args,stage=None):
    features = []
    for example_index, example in enumerate(examples):
        #source
        source_tokens = tokenizer.tokenize(example.source)[:args.max_source_length-2]
        source_tokens =[tokenizer.cls_token]+source_tokens+[tokenizer.sep_token]
        source_ids =  tokenizer.convert_tokens_to_ids(source_tokens) 
        source_mask = [1] * (len(source_tokens))
        padding_length = args.max_source_length - len(source_ids)
        source_ids+=[tokenizer.pad_token_id]*padding_length
        source_mask+=[0]*padding_length
 
        #target
        if stage=="test":
            target_tokens = tokenizer.tokenize("None")
        else:
            target_tokens = tokenizer.tokenize(example.target)[:args.max_target_length-2]
        target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]            
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_mask = [1] *len(target_ids)
        padding_length = args.max_target_length - len(target_ids)
        target_ids+=[tokenizer.pad_token_id]*padding_length
        target_mask+=[0]*padding_length   
   
        if example_index < 5:
            if stage=='train':
                logger.info("*** Example ***")
                logger.info("idx: {}".format(example.idx))

                logger.info("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokens]))
                logger.info("source_ids: {}".format(' '.join(map(str, source_ids))))
                logger.info("source_mask: {}".format(' '.join(map(str, source_mask))))
                
                logger.info("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokens]))
                logger.info("target_ids: {}".format(' '.join(map(str, target_ids))))
                logger.info("target_mask: {}".format(' '.join(map(str, target_mask))))
       
        features.append(
            InputFeatures(
                 example_index,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,
            )
        )
    return features

In [12]:
from transformers import RobertaTokenizer, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(args.model_name, do_lower_case=args.do_lower_case)

In [13]:
from transformers import RobertaConfig, RobertaModel

pretrained_model = args.model_name
beam_size = args.beam_size
output_dir = f'{args.save_dir}/{args.prefix}_{args.task}'
source_length = args.max_source_length
target_length = args.max_target_length

config = RobertaConfig.from_pretrained(pretrained_model)
encoder = RobertaModel.from_pretrained(pretrained_model, config = config)    
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model = Seq2Seq(encoder = encoder,decoder = decoder,config=config,
                beam_size=beam_size,max_length=target_length,
                sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)
model.load_state_dict(torch.load(Path(output_dir)/"checkpoint-best-bleu/pytorch_model.bin"))
model.to(device)

Some weights of the model checkpoint at neulab/codebert-cpp were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-cpp and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Seq2Seq(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [14]:
import pandas as pd

class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask      

class Example(object):
    """A single training/test example."""
    def __init__(self,
                 idx,
                 source,
                 target,
                 ):
        self.idx = idx
        self.source = source
        self.target = target

def get_preds(df: pd.DataFrame):
    ps = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        examples = [
            Example(idx, source = row.func_before, target = row.explain)
        ]
        eval_features = convert_examples_to_features(
            examples, tokenizer, args, stage='test'
        )
        source_ids = torch.tensor(eval_features[0].source_ids, dtype = torch.long).unsqueeze(0).to('cuda')
        source_mask = torch.tensor(eval_features[0].source_mask, dtype = torch.long).unsqueeze(0).to('cuda')

        with torch.no_grad():
            preds = model(source_ids = source_ids, source_mask = source_mask)  
            for pred in preds:
                t = pred[0].cpu().numpy()
                t = list(t)
                if 0 in t:
                    t = t[:t.index(0)]
                text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                ps.append(text)
    
    return ps

In [15]:
import evaluate
rouge = evaluate.load('rouge')

In [16]:
preds = get_preds(df_test)
references = []
for idx, row in df_test.iterrows():
    references.append(row.explain)

results = rouge.compute(predictions=preds, references=references)
results

  0%|          | 0/1076 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1064 > 512). Running this sequence through the model will result in indexing errors


{'rouge1': 0.6435438760949912,
 'rouge2': 0.6204438838732518,
 'rougeL': 0.6435576221589232,
 'rougeLsum': 0.6436332255105488}

In [17]:
for idx, row in df_test.head(3).iterrows():
    print('Code:', row.func_before)
    print('Original Comment:', row.explain)
    print('Generated Comment:', preds[idx])
    print('='*40)

Code: static ssize_t read_mem(struct file *file, char __user *buf,    size_t count, loff_t *ppos) {  phys_addr_t p = *ppos;  ssize_t read, sz;  void *ptr;   if (p != *ppos)   return 0;   if (!valid_phys_addr_range(p, count))   return -EFAULT;  read = 0; #ifdef __ARCH_HAS_NO_PAGE_ZERO_MAPPED  /* we don't have page 0 mapped on sparc and m68k.. */  if (p < PAGE_SIZE) {   sz = size_inside_page(p, count);   if (sz > 0) {    if (clear_user(buf, sz))     return -EFAULT;    buf += sz;    p += sz;    count -= sz;    read += sz;   }  } #endif  while (count > 0) {  unsigned long remaining;  sz = size_inside_page(p, count);  if (!range_is_allowed(p >> PAGE_SHIFT, count))  return -EPERM;  /*    * On ia64 if a page has been mapped somewhere as uncached, then    * it must also be accessed uncached by the kernel or data    * corruption may occur.    */   ptr = xlate_dev_mem_ptr(p);   if (!ptr)    return -EFAULT;  remaining = copy_to_user(buf, ptr, sz);   unxlate_dev_mem_ptr(p, ptr);  if (remaining)  r

In [18]:
df = []
for i, v in enumerate(zip(preds, references)):
    r_ = rouge.compute(predictions=[v[0]], references=[v[1]])
    df.append((i, r_['rouge1'], r_['rouge2'], r_['rougeL']))

In [19]:
df_ = pd.DataFrame(df, columns=['id', 'rouge1', 'rouge2', 'rougeL'])

In [20]:
df_.to_csv(f"{args.task}_cpp.tsv", sep='\t')